In [ ]:
from datasets import load_dataset
import torch
import numpy as np
import pickle
import json
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = load_dataset("boolq")

In [ ]:
input_pairs = [[row['question'], row['passage']] for row in tqdm(dataset['train'])]

In [ ]:
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("rycecorn/bert-fine-tuned-boolq")
model = BertModel.from_pretrained("rycecorn/bert-fine-tuned-boolq", output_attentions=True)

In [ ]:
def get_random_self_attention_matrix(input_sentence, attention_head):
    test = input_pairs[input_sentence]
    tokenized_input = tokenizer(test[0], test[1], truncation=True, padding=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
      outputs = model(**tokenized_input)
    attention_outputs = torch.stack(outputs.attentions)
    flattened_attention_matrices = attention_outputs.view(-1, attention_outputs.size(3), attention_outputs.size(4))
    flattened_attention_matrices = flattened_attention_matrices.numpy()
    
    selected_attention_head = flattened_attention_matrices[attention_head]
    return selected_attention_head

In [ ]:
img_num = 1

for interval in range(0, 1000):
    # RNG
    sentence_number = random.randint(0, 999)
    head_number = random.randint(0, 143)
    
    # Get random head
    head_check = get_random_self_attention_matrix(sentence_number, head_number)

    # Create and save figure
    file_name = f"{img_num}.png"
    plt.figure(figsize=(10, 8))
    plt.imshow(head_check, cmap='magma', interpolation='nearest')
    plt.savefig(".outputs/CNN/train_images/file_name")
    
    plt.close()
    
    # Increment file number
    img_num += 1